# cli

> Functions that implement the cli for the beetroot markdown backend

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp markdown.cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
#| export 
import argparse
import json 
from pathlib import Path
from typing import Dict, Protocol

In [ ]:
#| export
from beetroot.backend_config import BackendConfig
from beetroot.markdown.api import export_markdown_notebook
from beetroot.markdown.transformations import (
    EscapeEndLineSlashesWithinLatexMath,
    EscapeEqualsSignsAtLineStartWithinLatexMath,
    EscapeUnderscoresWithinLatexMath,
    MultiTransformer,
    ReplaceSingleDollarDelimiters,
    Unindent,
)


In [ ]:
#| export

class MarkdownBackendConfig(BackendConfig):
    def configure_arg_parser(self, parser: argparse.ArgumentParser): 
        parser.add_argument('output_dir', type=str, help="Output directory for the markdown and associated files")
        pass 

    def run(self, nb_json: Dict, args: argparse.Namespace):
        nb_path = Path(args.nb_filename)

        output_path = Path(args.output_dir)

        # TODO: add transformers dynamically based on cli args
        transformer = MultiTransformer([
            EscapeUnderscoresWithinLatexMath(),
            EscapeEndLineSlashesWithinLatexMath(),
            EscapeEqualsSignsAtLineStartWithinLatexMath(),
            ReplaceSingleDollarDelimiters()
        ])
        markdown, completions = export_markdown_notebook(
            nb_json, transformers_map={
                'markdown/source': transformer,
                'text/markdown/data/output': transformer,
                'text/latex/data/output': transformer,
                'text/html/data/output': Unindent(),
            }
        )
        # Run the completion
        for completion in completions:
            completion(output_path)

        # Write the markdown to an output file
        md_filename = output_path / nb_path.with_suffix('.md').name
        with open(md_filename, 'w') as md_file:
            md_file.write(markdown)    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()